Now that we have had extensive practice with using the `bcs` package to build chimeric PKSs, we can try using the main `RetroTide` package for the biosynthesis of targets given a custom list of starter and extender units. In this notebook, we will focus on attempting to reach only aliphatic targets. For this, we will use the more common, aliphatic starter and extender units.

As a reminder, if we wish to set custom starter and extender units, then we should import `bcs` first and change `bcs.starters` as well as `bcs.extenders` before eventually importing `retrotide` and `structureDB`

In [4]:
import bcs
from typing import Optional, List

In [5]:
def modify_bcs_starters_extenders(starter_codes: Optional[List[str]] = None, extender_codes: Optional[List[str]] = None):
    
    for key in list(bcs.starters.keys()):
        if key not in starter_codes:
            bcs.starters.pop(key, None) # removes key corresponding to the starter code that was not specified
            
    for key in list(bcs.extenders.keys()):
        if key not in extender_codes:
            bcs.extenders.pop(key, None) # same as above but for extenders

Here, we will constrain our list of starter and extender units to malonyl-CoA, methylmalonyl-CoA, hydroxyacetyl-CoA, methoxymalonyl-CoA, and allylmalonyl-CoA:

In [ ]:
starter_codes = ["Malonyl-CoA", 
                 "Methylmalonyl-CoA", 
                 "Hydroxymalonyl-CoA", 
                 "mxmal"]

extender_codes = ["Malonyl-CoA",
                  "Methylmalonyl-CoA",
                  "Hydroxymalonyl-CoA",
                  "mxmal",
                  "allylmal"]